In [8]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [11]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [12]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [13]:
answers = {}

In [14]:
# Some data structures that will be useful

In [15]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [16]:
len(allRatings)

200000

In [17]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [18]:
##################################################
# Read prediction                                #
##################################################

In [19]:
ratingsValid[:10]

[('u47877739', 'b50020691', 4),
 ('u44286663', 'b17025078', 5),
 ('u73526619', 'b22925633', 5),
 ('u39123859', 'b62815615', 0),
 ('u12786783', 'b99699446', 2),
 ('u99072969', 'b24832848', 4),
 ('u84518411', 'b73152433', 2),
 ('u50827374', 'b41385722', 3),
 ('u16732033', 'b77335595', 4),
 ('u24849922', 'b00310513', 5)]

In [20]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [21]:
### Question 1

In [22]:
newRatingsValid = [(user, book, 1) for user, book, _ in ratingsValid]
for user, book, _ in ratingsValid:
    newBook = random.choice(list(bookCount.keys()))
    while newBook in [b for b, _ in ratingsPerUser[user]]:
        newBook = random.choice(list(bookCount.keys()))
    newRatingsValid.append((user, newBook, 0))

In [23]:
random.shuffle(newRatingsValid)
newRatingsValid[-10:]

[('u93903821', 'b69804870', 1),
 ('u57052204', 'b71539352', 1),
 ('u92805325', 'b62685767', 0),
 ('u25959217', 'b22908875', 0),
 ('u08691057', 'b07766119', 1),
 ('u13761011', 'b85669891', 1),
 ('u07396386', 'b86037632', 1),
 ('u79880289', 'b48342589', 1),
 ('u47462976', 'b51416416', 1),
 ('u86445272', 'b81328113', 0)]

In [24]:
correct = 0
for _, book, predict in newRatingsValid:
    correct += predict == (book in return1)

acc1 = correct / len(newRatingsValid)
print(acc1)

0.7116


In [25]:
answers['Q1'] = acc1

In [26]:
assertFloat(answers['Q1'])

In [27]:
### Question 2

In [28]:
def solve(threshold):
    bookCount = defaultdict(int)
    totalRead = 0

    for _ ,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    mostPopular = [(bookCount[x], x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()

    return2 = set()
    count = 0

    for ic, i in mostPopular:
        count += ic
        return2.add(i)
        if count > totalRead * threshold: break

    correct = 0
    for _, book, predict in newRatingsValid:
        correct += predict == (book in return2)
    #print(correct, return2)
    acc = correct / len(newRatingsValid)
    return acc

In [29]:
l, r, eps = 0, 1, 1e-3
while r - l > eps:
    m1 = l + (r - l) / 3
    m2 = r - (r - l) / 3
    print(m1, m2)
    if solve(m1) < solve(m2):
        l = m1
    else:
        r = m2

best_threshold = l



0.3333333333333333 0.6666666666666667
0.5555555555555556 0.7777777777777778
0.7037037037037037 0.8518518518518519
0.654320987654321 0.7530864197530864
0.720164609053498 0.7860082304526749
0.6982167352537723 0.7421124828532236
0.6835848193872885 0.7128486511202561
0.7030940405426003 0.7226032616979119
0.696590966824163 0.7095971142610374
0.7052617317820793 0.7139324967399956
0.7110422417540235 0.7168227517259677


KeyboardInterrupt: 

In [30]:
acc2 = solve(best_threshold)
acc2

NameError: name 'best_threshold' is not defined

In [ ]:
answers['Q2'] = [best_threshold, acc2]

In [ ]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [ ]:
### Question 3/4

In [ ]:
def Jaccard(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2)) 

In [ ]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for user, item, _ in ratingsTrain:
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)

def solveQ3(threshold):
    ans = set()
    for user, book, _ in newRatingsValid:
        B = usersPerItem[book] #all users who read the book
        maxSim = 0
        for bprime in itemsPerUser[user]:
            if bprime == book:
                continue
            B2 = usersPerItem[bprime]
            sim = Jaccard(B, B2)
            maxSim = max(maxSim, sim)
            #print(maxSim)
        if maxSim > threshold:
            ans.add(book)

    correct = 0
    for _, book, predict in newRatingsValid:
        correct += predict == (book in ans)
    
    acc = correct / len(newRatingsValid)
    return acc


In [ ]:
l, r, eps = 0.006, 0.009, 1e-6
while r - l > eps:
    m1 = l + (r - l) / 3
    m2 = r - (r - l) / 3
    now1 = solveQ3(m1)
    now2 = solveQ3(m2)
    print(now1, l, now2, r)
    if now1 < now2:
        l = m1
    else:
        r = m2

best_threshold2 = l

0.69805 0.006 0.7005 0.009
0.69965 0.007 0.6999 0.009
0.70025 0.007666666666666666 0.69865 0.009
0.70045 0.007666666666666666 0.69975 0.008555555555555556
0.6998 0.007666666666666666 0.7005 0.00825925925925926
0.70065 0.007864197530864198 0.70025 0.00825925925925926
0.70045 0.007864197530864198 0.7005 0.008127572016460905
0.7005 0.007951989026063101 0.7004 0.008127572016460905
0.70065 0.007951989026063101 0.7005 0.008069044352994971
0.70065 0.007951989026063101 0.7005 0.008030025910684347
0.70065 0.007951989026063101 0.70065 0.008004013615810598
0.70045 0.007951989026063101 0.70065 0.007986672085894765
0.70065 0.00796355004600699 0.70065 0.007986672085894765
0.70065 0.00796355004600699 0.70065 0.007978964739265506
0.70045 0.00796355004600699 0.70065 0.007973826508179334
0.70065 0.007966975533397772 0.70065 0.007973826508179334
0.70065 0.007966975533397772 0.70065 0.007971542849918812
0.70045 0.007966975533397772 0.70065 0.007970020411078466
0.70065 0.00796799049262467 0.70065 0.0079700

In [ ]:
acc3 = solveQ3(best_threshold2)
acc3

0.70045

# --------------------------------- problem 4 -----------------------------------

In [ ]:
def solveQ4(threshold1, threshold2):
    ans1 = set()
    for user, book, _ in newRatingsValid:
        B = usersPerItem[book] #all users who read the book
        maxSim = 0
        for bprime in itemsPerUser[user]:
            if bprime == book:
                continue
            B2 = usersPerItem[bprime]
            sim = Jaccard(B, B2)
            maxSim = max(maxSim, sim)
            #print(maxSim)
        if maxSim > threshold1:
            ans1.add(book)

    bookCount = defaultdict(int)
    totalRead = 0

    for user,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    mostPopular = [(bookCount[x], x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()
    count = 0
    ans2 = set()
    for ic, i in mostPopular:
        count += ic
        ans2.add(i)
        if count > totalRead * threshold2: break

    global ret
    ret = set.union(ans1, ans2)
    correct = 0
    for _, book, predict in newRatingsValid:
        correct += predict == (book in ret)
    
    acc = correct / len(newRatingsValid)
    return acc

#idea is u just use 2 threshold instead of 1 for the function and find the maximum 
    


In [ ]:
best_threshold3 = scipy.optimize.fmin(lambda x: -solveQ4(x[0], x[1]), [0.699 ,0.6835848193872885 ])

Optimization terminated successfully.
         Current function value: -0.755950
         Iterations: 22
         Function evaluations: 58


In [ ]:
best_threshold3

array([0.70391484, 0.70815115])

In [ ]:
acc4 = solveQ4(best_threshold3[0], best_threshold3[1])

In [ ]:
acc4

0.75595

In [ ]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [ ]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [ ]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    if b in ret:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")


predictions.close()

In [ ]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [ ]:
assert type(answers['Q5']) == str

In [ ]:
##################################################
# Rating prediction                              #
##################################################

In [ ]:
### Question 6

In [ ]:
    
alpha = numpy.mean([r for _, _, r in ratingsTrain])
beta_user = defaultdict(float)
beta_book = defaultdict(float)
#referenced formula from 10/22 slides.  

for _ in range(50):
    alpha = sum(rating - (beta_user[user] + beta_book[book]) for user, book, rating in ratingsTrain) / len(ratingsTrain)
    for user, items in ratingsPerUser.items():
        beta_user[user] = sum(rating - (alpha + beta_book[book]) for book, rating in items) / (1 + len(items))
    for book, items in ratingsPerItem.items():
        beta_book[book] = sum(rating - (alpha + beta_user[user]) for user, rating in items) / (1 + len(items))

valid_error = [(rating - (alpha + beta_user.get(user, 0) + beta_book.get(book, 0))) ** 2 for user, book, rating in ratingsValid]
validMSE = numpy.mean(valid_error).item()

         

In [ ]:
validMSE

1.4618057928245451

In [ ]:
answers['Q6'] = validMSE

In [ ]:
assertFloat(answers['Q6'])

In [ ]:
### Question 7

In [ ]:
maxUser = str(max(beta_user, key=beta_user.get))
maxBeta = beta_user[maxUser]
minUser = str(min(beta_user, key=beta_user.get))
minBeta = beta_user[minUser]


In [ ]:
answers['Q7'] = [maxUser, minUser, maxBeta, minBeta]
answers['Q7']

['u18223169', 'u88024921', 1.8189836349785167, -3.5630493778377224]

In [ ]:
assert [type(x) for x in answers['Q7']] == [str, str, float, float]

In [ ]:
### Question 8

In [ ]:
import numpy

In [ ]:
def solve(lambda_reg):
    alpha = numpy.mean([r for _, _, r in ratingsTrain])
    beta_user = defaultdict(float)
    beta_book = defaultdict(float)
    #referenced formula from 10/22 slides.  

    for _ in range(10):
        alpha = sum(rating - (beta_user[user] + beta_book[book]) for user, book, rating in ratingsTrain) / len(ratingsTrain)
        for user, items in ratingsPerUser.items():
            beta_user[user] = sum(rating - (alpha + beta_book[book]) for book, rating in items) / (lambda_reg + len(items))
        for book, items in ratingsPerItem.items():
            beta_book[book] = sum(rating - (alpha + beta_user[user]) for user, rating in items) / (lambda_reg + len(items))

    valid_error = [(rating - (alpha + beta_user.get(user, 0) + beta_book.get(book, 0))) ** 2 for user, book, rating in ratingsValid]
    validMSE = numpy.mean(valid_error).item()
    return validMSE

In [ ]:
print(solve(0.6))

NameError: name 'ratingsTrain' is not defined

In [ ]:
l, r, eps = 0, 100, 1e-4

while(r-l > eps):
    m1 = l + (r-l)/3
    m2 = r - (r-l)/3
    print(m1, m2)
    if solve(m1) < solve(m2):
        r = m2
    else:
        l = m1

print(l, solve(l))

lamb = l
validMSE = solve(l)

33.333333333333336 66.66666666666666
22.222222222222218 44.44444444444444
14.814814814814815 29.629629629629626
9.876543209876543 19.75308641975308
6.584362139917694 13.168724279835388
4.38957475994513 8.779149519890257
2.9263831732967525 5.852766346593505
4.8773052888279205 6.828227404359089
4.226997916984198 5.527612660671644
3.793459669088383 4.660536164880013
4.371510666282803 4.9495616634772235
4.17882700055133 4.564194332014276
4.050371223397014 4.307282777705645
4.221645592936102 4.3929199624751885
4.3358285059621595 4.450011418988218
4.297767534953474 4.373889476970846
4.348515496298389 4.399263457643303
4.331599509183417 4.36543148341336
4.320322184440102 4.342876833926732
4.335358617431188 4.350395050422274
4.3303464731008265 4.34037076176155
4.337029332207976 4.343712191315125
4.341484571612742 4.345939811017508
4.339999491811153 4.342969651414331
4.341979598213271 4.34395970461539
4.341319562745899 4.342639633680644
4.342199610035729 4.34307965732556
4.341906260939119 4.342

In [ ]:
answers['Q8'] = (lamb, validMSE)

In [ ]:
assertFloat(answers['Q8'][0])
assertFloat(answers['Q8'][1])

In [ ]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    prediction = alpha + beta_user.get(u, 0) + beta_book.get(b, 0)
    predictions.write(u + ',' + b + ',' + str(prediction) + '\n')
    
predictions.close()

NameError: name 'alpha' is not defined

In [ ]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()